# Processing Census 2011 Web Data

## TO DO LIST
- Check all names from the pdfs and dict_cols
- Make a script that combines everything into a one script
- 

In [62]:
import os
import re
import PyPDF2
from collections import Counter
import pandas as pd
from config import Config
os.getcwd()

'/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/utils/parsepdfs'

In [63]:
census2011 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/webtestapplication/userfiles/image/Census2011'


In [64]:
def check_pdf(file):
    df_file_obj = open(file, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(df_file_obj, strict=False)
    num_pages = pdf_reader.getNumPages()
    pages = ''
    for page in range(num_pages):
        page_obj = pdf_reader.getPage(page)
        pages = pages + page_obj.extractText()
    pages = pages.split("\n")
    pages = [element.replace(",", "") for element in pages]
    check_points = []
    for key, element in enumerate(pages):
        if element.isspace() or not element.strip():
            pages[key] = float('nan')
        if key > 6:
            try:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                digits_check = [int(digit) for digit in digits_check]
                geoname_check = not string_check.isdigit()
                values_check_int = all([str(digit).isdigit() for digit in digits_check])
                geoname_values_check = geoname_check and values_check_int
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    check_points.append(key)
            except ValueError:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                geoname_check = not check_float(string_check)
                values_check_float = all([check_float(digit) for digit in digits_check])
                geoname_values_check = geoname_check and values_check_float
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    check_points.append(key)
            else:
                pass
    out = [j-i for i,j in zip(check_points[:-1], check_points[1:])]
    return Counter(out).most_common(1)

In [71]:
output = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/bbs/'
Kushtia = '../../data/bbs/webtestapplication/userfiles/image/Census2011/Khulna/Kushtia/Kushtia_C12.pdf'
out_test = check_pdf(file=Kushtia)
out_test[0][0]

15

In [27]:
def set_wd(path):
    os.chdir(path)


def check_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False


def get_pdf_files():
    output = {'file': [], 'program': []}
    for path, subdirs, files in os.walk('.'):
        for name in files:
            if bool(re.search(r'_C[0-9][0-9].pdf', name)):
                output['file'].append(os.path.join(path, name))
                output['program'].append(name[-7:-4])
    return pd.DataFrame.from_dict(output)

def parse_files(file, ncol_before, ncol_after, col_names):
    df_file_obj = open(file, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(df_file_obj, strict=False)
    num_pages = pdf_reader.getNumPages()
    pages = ''
    subset = {}
    for page in range(num_pages):
        page_obj = pdf_reader.getPage(page)
        pages = pages + page_obj.extractText()
    pages = pages.split("\n")
    pages = [element.replace(",", "") for element in pages]
    for key, element in enumerate(pages):
        if element.isspace() or not element.strip():
            pages[key] = float('nan')
        if key > 6:
            try:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                digits_check = [int(digit) for digit in digits_check]
                geoname_check = not string_check.isdigit()
                values_check_int = all([str(digit).isdigit() for digit in digits_check])
                geoname_values_check = geoname_check and values_check_int
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key - ncol_before:key + ncol_after]
            except ValueError:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                geoname_check = not check_float(string_check)
                values_check_float = all([check_float(digit) for digit in digits_check])
                geoname_values_check = geoname_check and values_check_float
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key - ncol_before:key + ncol_after]
            else:
                pass
    subset = pd.DataFrame.from_dict(subset, orient='index', columns=col_names)
    df_file_obj.close()
    return subset

def write_data(input_files, output_dir):
    for subset in input_files['program'].unique():
        tmp = input_files[input_files['program'] == subset]
        out_list = []
        for key, values in enumerate(tmp.values):
            data = parse_files(values[0],
                           ncol_before=Config.parse_parameters[values[1]]['ncol_before'],
                           ncol_after=Config.parse_parameters[values[1]]['ncol_after'],
                           col_names=Config.cols_dict[values[1]])
            out_list.append(data)
        program_df = pd.concat(out_list)
        program_df.to_csv(os.path.join(output_dir,subset+".csv"), index=False, index_label=None)

def main():
    set_wd(path=census2011)
    out = get_pdf_files()
    write_data(input_files=out,output_dir=output)
    return out

In [ ]:
main()

In [ ]:
t1 = parse_files(file=baruna_files[0], 
                     ncol_before=6, 
                     ncol_after=5, 
                     col_names = cols_dict['C01'])
print(t1['ZL'].value_counts())
t1.head()


In [ ]:
t2 = parse_files(file=baruna_files[1], ncol_before=6, ncol_after=9, col_names = cols_dict['C02'])
print(t2['ZL'].value_counts())
t2.head()

In [ ]:
t3 = parse_files(file=baruna_files[2], ncol_before=6, ncol_after=11, col_names = cols_dict['C03'])
print(t3['ZL'].value_counts())
t3.head()

In [ ]:
t4 = parse_files(file=baruna_files[3], ncol_before=6, ncol_after=12, col_names = cols_dict['C04'])
print(t4['ZL'].value_counts())
t4.head()

In [ ]:
t5 = parse_files(file=baruna_files[4], ncol_before=6, ncol_after=11, col_names = cols_dict['C05'])
print(t5['ZL'].value_counts())
t5.head()

In [ ]:
print(cols_dict['C06'])
print(len(cols_dict['C06']))
t6 = parse_files(file=baruna_files[5], ncol_before=6, ncol_after=10, col_names = cols_dict['C06'])
print(t6['ZL'].value_counts())
t6.head()

In [ ]:
print(cols_dict['C07'])
print(len(cols_dict['C07']))
t7 = parse_files(file=baruna_files[6], ncol_before=6, ncol_after=13, col_names = cols_dict['C07'])
print(t7['ZL'].value_counts())
t7.head()

In [ ]:
print(cols_dict['C08'])
print(len(cols_dict['C08']))
t8 = parse_files(file=baruna_files[7], ncol_before=6, ncol_after=13, col_names = cols_dict['C08'])
print(t8['ZL'].value_counts())
t8.head()

In [ ]:
print(cols_dict['C09'])
print(len(cols_dict['C09']))
t9 = parse_files(file=baruna_files[8], ncol_before=6, ncol_after=9, col_names = cols_dict['C09'])
print(t9['ZL'].value_counts())
t9.head()

In [ ]:
print(cols_dict['C10'])
print(len(cols_dict['C10']))
t10 = parse_files(file=baruna_files[9], ncol_before=6, ncol_after=9, col_names=cols_dict['C10'])
print(t10['ZL'].value_counts())
t10.head()

In [ ]:
print(cols_dict['C11'])
print(len(cols_dict['C11']))
t11 = parse_files(file=baruna_files[10], ncol_before=6, ncol_after=10, col_names=cols_dict['C11'])
print(t11['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C12'])
print(len(cols_dict['C12']))
t12 = parse_files(file=baruna_files[11], ncol_before=6, ncol_after=9, col_names=cols_dict['C12'])
print(t12['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C12_e'])
print(len(cols_dict['C12_e']))
t12_e = parse_files(file=baruna_files[12], ncol_before=6, ncol_after=9, col_names=cols_dict['C12_e'])
print(t12_e['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C13'])
print(len(cols_dict['C13']))
t13 = parse_files(file=baruna_files[13], ncol_before=6, ncol_after=7, col_names=cols_dict['C13'])
print(t13['ZL'].value_counts())
t13.head()

In [ ]:
print(cols_dict['C14'])
print(len(cols_dict['C14']))
t14 = parse_files(file=baruna_files[14], ncol_before=6, ncol_after=10, col_names=cols_dict['C14'])
print(t14['ZL'].value_counts())
t14.head()

In [ ]:
print(cols_dict['C15'])
print(len(cols_dict['C15']))
t15 = parse_files(file=baruna_files[15], ncol_before=6, ncol_after=9, col_names=cols_dict['C15'])
print(t15['ZL'].value_counts())
t15.head()

In [9]:
    cols_dict = {'C01': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community', 'Area in Acres',
                         'Total Households', 'Population-Total', 'Population-In Households'],
                 'C02': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Households-Total', 'Households-General', 'Households-Institutional',
                         'Households-Others', 'Population-Both', 'Population-Male', 'Population-Female', 'Sex Ratio'],
                 'C03': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'General Households', 'Percentage of Households comprising-1 person',
                         'Percentage of Households comprising-2 persons',
                         'Percentage of Households comprising-3 persons',
                         'Percentage of Households comprising-4 persons',
                         'Percentage of Households comprising-5 persons',
                         'Percentage of Households comprising-6 persons',
                         'Percentage of Households comprising-7 persons',
                         'Percentage of Households comprising-8+ persons', 'Average size of Household'],
                 'C04': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community', 'All ages',
                         'Percentage of population in the age group 0-4',
                         'Percentage of population in the age group 5-9',
                         'Percentage of population in the age group 10-14',
                         'Percentage of population in the age group 15-19',
                         'Percentage of population in the age group 20-24',
                         'Percentage of population in the age group 25-29',
                         'Percentage of population in the age group 30-49',
                         'Percentage of population in the age group 50-59',
                         'Percentage of population in the age group 60-64',
                         'Percentage of population in the age group 65+', ],
                 'C05': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Total Male', '% Male-Never married', '% Male-Married', '% Male-Widowed',
                         '% Male-Divorced/Separated',
                         'Total Female', '% Female-Never married', '% Female-Married', '% Female-Widowed',
                         '% Female-Divorced/Separated'],
                 'C06': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Literate (can write a letter)-Both-Yes', 'Literate (can write a letter)-Both-No',
                         'Literate (can write a letter)-Male-Yes', 'Literate (can write a letter)-Male-No',
                         'Literate (can write a letter)-Female-Yes', 'Literate (can write a letter)-Female-No',
                         'Literacy Rate (%)-Both', 'Literacy Rate (%)-Male', 'Literacy Rate (%)-Female', ],
                 'C07': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Population aged 3-5 years-Attending school-Male',
                         'Population aged 3-5 years-Attending school-Female',
                         'Population aged 3-5 years-Not attending school-Male',
                         'Population aged 3-5 years-Not attending school-Female',
                         'Population aged 6-10 years-Attending school-Male',
                         'Population aged 6-10 years-Attending school-Female',
                         'Population aged 6-10 years-Not attending school-Male',
                         'Population aged 6-10 years-Not attending school-Female',
                         'Population aged 11-14 years-Attending school-Male',
                         'Population aged 11-14 years-Attending school-Female',
                         'Population aged 11-14 years-Not attending school-Male',
                         'Population aged 11-14 years-Not attending school-Female', ],
                 'C08': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Population aged 15-19 years-Attending school-Male',
                         'Population aged 15-19 years-Attending school-Female',
                         'Population aged 15-19 years-Not attending school-Male',
                         'Population aged 15-19 years-Not attending school-Female',
                         'Population aged 20-24 years-Attending school-Male',
                         'Population aged 20-24 years-Attending school-Female',
                         'Population aged 20-24 years-Not attending school-Male',
                         'Population aged 20-24 years-Not attending school-Female',
                         'Population aged 25-29 years-Attending school-Male',
                         'Population aged 25-29 years-Attending school-Female',
                         'Population aged 25-29 years-Not attending school-Male',
                         'Population aged 25-29 years-Not attending school-Female', ],

                 'C09': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Total Population', 'Type of disability (%)-All', 'Type of disability (%)-Speech',
                         'Type of disability (%)-Vision', 'Type of disability (%)-Hearing',
                         'Type of disability (%)-Physical', 'Type of disability (%)-Mental',
                         'Type of disability (%)-Autism', ],
                 'C10': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Population aged 7+ and not attending school-Both',
                         'Population aged 7+ and not attending school-Male',
                         'Population aged 7+ and not attending school-Female',
                         'Employment Status-Employed-Male', 'Employment Status-Employed-Female',
                         'Employment Status-Looking for work-Male', 'Employment Status-Looking for work-Female',
                         'Employment Status-Household work-Male', 'Employment Status-Household work-Female',
                         'Employment Status-Do not work-Male', 'Employment Status-Do not work-Female', ],
                 'C11': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Population aged 7+, not attending school and employed-Both',
                         'Population aged 7+, not attending school and employed-Male',
                         'Population aged 7+, not attending school and employed-Female',
                         'Field of Activity-Agriculture-Male', 'Field of Activity-Agriculture-Female',
                         'Field of Activity-Industry-Male', 'Field of Activity-Industry-Female',
                         'Field of Activity-Service-Male', 'Field of Activity-Service-Female', ],
                 'C12_e': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                           'Administrative Unit Residence Community',
                           'Ethnic-Households', 'Ethnic-Population-Both', 'Ethnic-Population-Male',
                           'Ethnic-Population-Female',
                           'Ethnic Population in Main Groups-Rakhain', 'Ethnic Population in Main Groups-Chakma',
                           'Ethnic Population in Main Groups-Marma', 'Ethnic Population in Main Groups-Others', ],
                 'C12': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Ethnic-Households', 'Ethnic-Population-Both', 'Ethnic-Population-Male',
                         'Ethnic-Population-Female',
                         'Ethnic Population in Main Groups-Malpahari', 'Ethnic Population in Main Groups-Chakma',
                         'Ethnic Population in Main Groups-Dalu', 'Ethnic Population in Main Groups-Others', ],
                 'C13': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Total', 'Muslim', 'Hindu', 'Christian', 'Buddhist', 'Others'],
                 'C14': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Number of Households', 'Type of Structure (%)-Pucka', 'Type of Structure (%)-Semi-pucka',
                         'Type of Structure (%)-Kutcha', 'Type of Structure (%)-Jhupri',
                         'Toilet Facility (%)-Sanitary (water-sealed)',
                         'Toilet Facility (%)-Sanitary (non water-sealed)',
                         'Toilet Facility (%)-Non-sanitary', 'Toilet Facility (%)-None', ],
                 'C15': ['ZL', 'UZ', 'UN/WA', 'MZ/MH', 'Vill', 'RMO',
                         'Administrative Unit Residence Community',
                         'Number of Households',
                         'Source of Drinking Water (%)-Tap', 'Source of Drinking Water (%)-Tube-Well',
                         'Source of Drinking Water (%)-Other',
                         'Electricity Connection (%)',
                         'Housing Tenancy (%)-Owned', 'Housing Tenancy (%)-Rented', 'Housing Tenancy (%)-Rent free'],
                 }
parse_parameters = {'C01': {'ncol_before': 6, 'ncol_after': 5},
                        'C02': {'ncol_before': 6, 'ncol_after': 9},
                        'C03': {'ncol_before': 6, 'ncol_after': 11},
                        'C04': {'ncol_before': 6, 'ncol_after': 12},
                        'C05': {'ncol_before': 6, 'ncol_after': 11},
                        'C06': {'ncol_before': 6, 'ncol_after': 10},
                        'C07': {'ncol_before': 6, 'ncol_after': 13},
                        'C08': {'ncol_before': 6, 'ncol_after': 13},
                        'C09': {'ncol_before': 6, 'ncol_after': 9},
                        'C10': {'ncol_before': 6, 'ncol_after': 12},
                        'C11': {'ncol_before': 6, 'ncol_after': 10},
                        'C12': {'ncol_before':6, 'ncol_after':18},
                        'C13': {'ncol_before': 6, 'ncol_after': 7},
                        'C14': {'ncol_before': 6, 'ncol_after': 10},
                        'C15': {'ncol_before': 6, 'ncol_after': 9},
                           }

In [10]:
for key,values in cols_dict.items():
    print(key, len(values))

C01 11
C02 15
C03 17
C04 18
C05 17
C06 16
C07 19
C08 19
C09 15
C10 18
C11 16
C12_e 15
C12 15
C13 13
C14 16
C15 15


In [13]:
for key,values in parse_parameters.items():
    print(f"{key} {parse_parameters[key]['ncol_after']+parse_parameters[key]['ncol_before']}")

C01 11
C02 15
C03 17
C04 18
C05 17
C06 16
C07 19
C08 19
C09 15
C10 15
C11 16
C12 24
C13 13
C14 16
C15 15
